In [45]:
import pathlib
import pandas as pd
from scipy import sparse
import numpy as np
import pickle

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [5]:
from sentence_transformers import SentenceTransformer, util
import time

class Index(object):
    def __init__(self, corpus: list, doc_ids: list, model_name: str = 'SeyedAli/Multilingual-Text-Semantic-Search-Siamese-BERT-V1'):
        self.model = SentenceTransformer(model_name)
        self.corpus = corpus
        self.doc_ids = doc_ids
        
        # Preprocess the corpus: split each document into sentences
        self.sentences = []
        self.mapping = []
        for doc_id, document in enumerate(corpus):
            
            # Split the document into sentences based on "."
            substrings = document.split(".")
            substrings = [s.strip() for s in substrings if s.strip()]  #
            
            self.sentences.extend(substrings)
            self.mapping.extend([doc_id] * len(substrings))
        
        print("-- -- Encoding corpus...")
        start_time = time.time()
        self.index = self.model.encode(self.sentences)
        print("-- -- Corpus encoded in {} minutes".format((time.time()-start_time)/60))
    
    def retrieve(self, query, topk=5):
        # Encode query
        query_emb = self.model.encode(query)
        
        # Compute dot score between query and all sentence embeddings
        scores = util.dot_score(query_emb, self.index)[0].cpu().tolist()
        
        # Combine sentences & scores
        sentence_score_pairs = list(zip(self.sentences, scores, self.mapping))
        
        # Sort by decreasing score
        sentence_score_pairs = sorted(sentence_score_pairs, key=lambda x: x[1], reverse=True)[:topk]
        
        # Retrieve the topk results with original document mapping
        results = []
        for sentence, score, doc_id in sentence_score_pairs:
            results.append({
                "document_id": self.doc_ids[doc_id],
                "sentence": sentence,
                "score": score,
                "original_document": self.corpus[doc_id]
            })
        
        return results

/home/lbartolome/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
def get_doc_top_tpcs(doc_distr, topn=2):
    sorted_tpc_indices = np.argsort(doc_distr)[::-1]
    top = sorted_tpc_indices[:topn].tolist()
    top_weight = [(k, doc_distr[k]) for k in top]
    return top_weight

def get_doc_main_topc(doc_distr):
    sorted_tpc_indices = np.argsort(doc_distr)[::-1]
    top = sorted_tpc_indices[:1][0]
    return top

In [10]:
path_orig_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_en_tr.parquet")
path_orig_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_es_tr.parquet")
path_source = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/multi_blade_filtered/df_1.parquet")
path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/MULTI_BLADE_FILTERED/poly_rosie_v2_1_20")

In [12]:
raw = pd.read_parquet(path_source)
raw.head(2)

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang
0,0,implant_port information technology easy child prolonged treatment blood draw certain chronic illness place port noticeable information technology interfere_daily activity healthcare_provider discuss pro_con implant_port,puerto_implantado facilitar hijo recibir tratamiento prolongado extracción sangre enfermedad crónico puerto perceptible interferir mayoría actividad diario proveedor_atención médico discutir_pro puerto_implantado,EN_657272_107210-27,"Implanted ports make it easier for you or your child to get prolonged treatments or blood draws for certain chronic illnesses. Once in place, a port is barely noticeable. It shouldn’t interfere with most daily activities. Your healthcare provider can discuss the pros and cons of using an implanted port.","Los puertos implantados facilitan que usted o su hijo reciban tratamientos prolongados o extracciones de sangre para ciertas enfermedades crónicas. Una vez en su lugar, un puerto es apenas perceptible. No debe interferir con la mayoría de las actividades diarias. Su proveedor de atención médica puede discutir los pros y los contras del uso de un puerto implantado.",EN
1,1,life_expectancy birth reach record high year total population disparity life_expectancy birth remain race sex population population increase life_expectancy past_decade additional information life_expectancy available,esperanza_vida nacer alcanzar_máximo precedente año población unidos disparidad esperanza_vida nacer mantener población raza sexo población experimentar aumento esperanza_vida decenio obtener él esperanza_vida,EN_403516_60988-2,"Life expectancy at birth reached a record high of 77.9 years in 2005 for the total U.S. population. Disparities in life expectancy at birth remain among the race/sex populations, although all populations have had increases in life expectancy during the past decade. Additional information regarding life expectancy is available at http://www.cdc.gov/nchs/deaths.htm.","La esperanza de vida al nacer alcanzó un máximo sin precedentes de 77,9 años en 2005 para la población total de los Estados Unidos. Las disparidades en la esperanza de vida al nacer se mantienen entre las poblaciones de raza y sexo, aunque todas las poblaciones han experimentado un aumento de la esperanza de vida durante el último decenio. Puede obtenerse más información sobre la esperanza de vida en http://www.cdc.gov/nchs/deaths.htm.",EN


In [37]:
# get topic keys in Engish
with open(path_model / "mallet_output" / "keys_ES.txt", 'r') as file:
    lines = file.readlines()
topic_keys_es = [line.strip() for line in lines]
for id_el, el in enumerate(topic_keys_es):
    print(f"TOPIC {id_el}: {el}") 

TOPIC 0: médico comenzar mayo letra clinic apellido buscar afección minnesota rochester mostrar búsqueda trasplante enfermedad áreas área florida mn encontrar
TOPIC 1: síntoma medicamento dolor tomar causar médico grave problema tratar infección tratamiento persona sentir signo leve fiebre piel pérdida niño
TOPIC 2: paciente tratamiento clínico terapia recibir ensayo riesgo mes grupo medicamento diagnóstico año evidencia enfermedad evaluar dosis mejorar tratar alto
TOPIC 3: cáncer tumor tratamiento célula tipo cirugía detección mama prueba riesgo radiación tejido glándula radioterapia canceroso cuello examen próstata quimioterapia
TOPIC 4: trastorno síndrome gen célula genético proteína cerebro causar persona mutación enfermedad cerebral afectar llamado ácido tipo sistema función afección
TOPIC 5: bebé mujer embarazo embarazado riesgo sexual nacimiento parto nacido semana problema nacer madre materno aborto mes defecto anticonceptivo útero
TOPIC 6: infección prueba persona caso enferme

In [38]:
# get topic keys in Engish
with open(path_model / "mallet_output" / "keys_EN.txt", 'r') as file:
    lines = file.readlines()
topic_keys_en = [line.strip() for line in lines]
for id_el, el in enumerate(topic_keys_en):
    print(f"TOPIC {id_el}: {el}") 

TOPIC 0: doctor clinic mayo condition search minnesota rochester medical focus disease transplant surname care florida result center treat health location
TOPIC 1: symptom pain technology medication medicine doctor treat severe feel infection treatment child day people sign skin include fever mild
TOPIC 2: patient treatment study therapy clinical drug receive organization treat risk month trial follow evidence health rate effective day improve
TOPIC 3: cancer tumor treatment breast cell radiation technology surgery therapy screening risk tissue thyroid stage treat gland prostate remove hormone
TOPIC 4: disorder syndrome gene brain cell condition protein genetic affect mutation acid disease technology people function change development seizure develop
TOPIC 5: baby woman pregnancy birth technology pregnant health infant risk organization week mother bear sex sexual month period delivery newborn
TOPIC 6: infection hiv person report virus tuberculosis disease laboratory outbreak health te

In [15]:
thetas = sparse.load_npz(path_model / "mallet_output" / "thetas_EN.npz")
raw["thetas"] = list(thetas.toarray())
raw.loc[:, "top_k"] = raw["thetas"].apply(get_doc_top_tpcs)
raw.loc[:, "main_topic"] = raw["thetas"].apply(get_doc_main_topc)

In [17]:
raw[["text","top_k","main_topic"]].head(2)

,text,top_k,main_topic
0,"Implanted ports make it easier for you or your child to get prolonged treatments or blood draws for certain chronic illnesses. Once in place, a port is barely noticeable. It shouldn’t interfere with most daily activities. Your healthcare provider can discuss the pros and cons of using an implanted port.","[(9, 0.6530612111091614), (19, 0.1428571492433548)]",9
1,"Life expectancy at birth reached a record high of 77.9 years in 2005 for the total U.S. population. Disparities in life expectancy at birth remain among the race/sex populations, although all populations have had increases in life expectancy during the past decade. Additional information regarding life expectancy is available at http://www.cdc.gov/nchs/deaths.htm.","[(10, 0.380952388048172), (15, 0.3333333432674408)]",10


In [22]:
df_en = raw[raw['doc_id'].str.startswith("EN")].copy()
df_es = raw[raw['doc_id'].str.startswith("ES")].copy()

In [39]:
df_en["counts"] = df_es.apply(lambda x: len(list(set(x["lemmas"].split()) - set(topic_keys_en[x["main_topic"]].split()))), axis=1)
df_es["counts"] = df_es.apply(lambda x: len(list(set(x["lemmas"].split()) - set(topic_keys_es[x["main_topic"]].split()))), axis=1)

In [36]:
df_es_5 = df_es[df_es.main_topic == 5]
df_es_5[["text", "lemmas", "top_k", "main_topic", "counts"]].head(5)

,text,lemmas,top_k,main_topic,counts
18,Esa misma contaminación atmosférica que causa el cambio climático está vinculada con siete millones de muertes al año.,contaminación_atmosférico causar cambio_climático vinculado millón muerte año,"[(5, 0.8571428656578064), (19, 0.0357142873108387)]",5,7
28,"Si su hijo tiene una cita de control, anote la fecha, la hora y el propósito de esa visita.",hijo control anote_fecha hora visita,"[(5, 0.6538461446762085), (3, 0.19230769574642181)]",5,5
31,Qué sucedería si no se hace la prueba o el procedimiento.,suceder prueba,"[(5, 0.4810126721858978), (18, 0.40506330132484436)]",5,2
32,"Este estudio analiza una nueva tecnología para medir la actividad cardíaca fetal y encontrar posibles problemas en forma temprana. Estos problemas incluyen arritmias y trastornos del sistema de conducción en el feto, como síndrome de Brugada. Para participar en este estudio, debe tener al menos 18 años y estar embarazada, y debe tener uno de cinco complicaciones de alto riesgo del embarazo: feto con una malformación congénita importante, hidropesía o gastrosquisis, mortinato previo sin causa que lo explique, o gemelos que comparten una placenta. Este estudio se lleva a cabo en Madison y Milwaukee (Wisconsin).",analizar_tecnología medir actividad cardíacar_fetal encontrar problema forma temprano problema arritmia trastorno sistema_conducción feto síndrome_brugada participar año embarazado complicación alto riesgo embarazo feto_malformación congénita importante_hidropesía gastrosquisis_mortinato previo causa_gemelo compartir_placenta cabo_madison milwaukee_wisconsin,"[(5, 0.3529411852359772), (1, 0.23529411852359772)]",5,25
49,"¿Qué ocurre en una prueba genética del PTEN: Si se va a hacer una prueba genética del PTEN porque tiene cáncer, el tipo de muestra que se utilice dependerá de su tipo de cáncer:",prueba genético_pten prueba genético_pten cáncer tipo muestra depender tipo cáncer,"[(5, 0.30000001192092896), (3, 0.30000001192092896)]",5,6


In [41]:
df_en_5 = df_en[df_en.main_topic == 5]
df_en_5[["text", "lemmas", "top_k", "main_topic", "counts"]].head(5)

,text,lemmas,top_k,main_topic,counts
42,Proper hand hygiene is key to stopping the spread of stomach bugs. Remind everyone in your family about the two most important times to wash up with soap and water:\n- After using the bathroom or changing diapers\n- Before eating or handling food.,proper hand_hygiene key stop spread stomach_bug remind family important time wash_soap bathroom change eat handle food,"[(5, 0.5098039507865906), (1, 0.4117647111415863)]",5,17.0
63,"Healthy People 2020 includes a public health infrastructure objective (PHI 7.3) to increase the percentage of population-based health objectives for which national data are available by socioeconomic status (10). Public heath surveillance data often lack information on the SES of individual persons, making it difficult to describe and monitor health disparities based on SES. When residential address data are available, it is possible to geocode the address, link it to census tract SES data and conduct analyses. The EIP, recognizing the potential to use geocoded addresses to examine possible SES-related disparities at a broad interstate level, formed a health equity workgroup to explore standardizing its use in all EIP states (11). This analysis demonstrates that multisite analyses using census tract SES can provide national-level data that are relevant to prevention efforts.",healthy people include public health infrastructure objective phi increase percentage population base health objective national datum available socioeconomic_status public heath surveillance datum lack information ses individual person information technology difficult describe monitor health disparity base ses residential address datum available information technology possible geocode_address link information technology census_tract ses datum conduct analysis eip recognize potential geocode_address examine possible ses disparity broad interstate level form health equity workgroup explore standardize eip state analysis demonstrate multisite analysis census_tract ses national level datum relevant prevention effort,"[(5, 0.35555556416511536), (10, 0.17777778208255768)]",5,27.0
97,"Most index patients (88.4%, 152 of 172) and household contacts (78.7%, 140 of 178) with confirmed cases reported COVID-19–compatible symptoms. Of those with known SARS-CoV-2 infection history, eleven (6.1%) of 181 index patients and nine (4.7%) of 192 household contacts with confirmed or probable COVID-19 reported a previous SARS-CoV-2 infection.",index patient household contact confirm case report compatible symptom know sars infection history index patient household contact confirm_probable report previous sars infection,"[(5, 0.5), (19, 0.21052631735801697)]",5,NaN
113,"Billing Disputes: A review of billing disputes is another method of identifying risk and quality deficiencies. Patients often refuse to pay bills because they believe that the care received was substandard and therefore not deserving of payment. Some patients also make accusations of poor quality care simply to justify their refusal to pay for care. This compromises the cost-effectiveness of billing disputes as a risk identification tool. Consequently, detailed clinical review of all cases resulting in a billing dispute should probably be reserved for cases involving large sums of money or significant accusations of substandard quality.",billing_dispute review billing_dispute method identify risk quality deficiency patient refuse pay_bill believe care receive substandard deserve payment patient accusation poor quality care justify refusal pay care compromise cost_effectiveness billing_dispute risk identification tool detailed clinical review case result billing_dispute reserve case involve large sum money significant accusation substandard quality,"[(5, 0.5833333134651184), (19, 0.1666666716337204)]",5,NaN
170,"In a major advancement in the field of gene therapy for rare and devastating diseases, researchers at Children’s Hospital of Philadelphia (CHO

In [40]:
index_es_5 = Index(corpus=df_es_5.text.values.tolist(), doc_ids=df_es_5.doc_id.values.tolist())

-- -- Encoding corpus...
-- -- Corpus encoded in 1.5480829199155173 minutes


In [44]:
query = "¿Cómo puedo ayudar a mi bebé a dormir?"#"How can I help my baby to sleep?"
results = index_es_5.retrieve(query, topk=10)

for result in results:
    print(f"Document ID: {result['document_id']}")
    print(f"Sentence: {result['sentence']}")
    print(f"Score: {result['score']}")
    print(f"Original Document: {result['original_document']}")
    print("\n")

Document ID: ES_280552_40484-84
Sentence: Dormir puede ayudar a mejorar su estado de ánimo y hacerle sentir menos estresado
Score: 0.7727817893028259
Original Document: A continuación se incluyen algunas cosas que debe hacer:\n- No abuse del alcohol ni de las drogas. Tener un problema de abuso de sustancias hace que sea más difícil tratar la esquizofrenia. Si tiene usted un problema de abuso de sustancias, es necesario tratar ambos problemas para ayudar a su recuperación.\n- Haga ejercicio y actividad física . El ejercicio puede ayudar a aliviar la ansiedad, la depresión y el estrés. El ejercicio y la actividad reducen además las posibilidades de tener otras enfermedades, como enfermedad cardíaca o diabetes.\n- Alivie el estrés . La reducción del estrés puede significar un menor número de recaídas.\n- Duerma lo suficiente . Esto es muy importante. Dormir puede ayudar a mejorar su estado de ánimo y hacerle sentir menos estresado.\n- Siga una dieta equilibrada . Esto ayuda al cuerpo a ma

In [48]:
#index_all_es = pickle.load(open('index_store_es_all.pkl', 'rb'))
index_all_es = Index(corpus=df_es.text.values.tolist(), doc_ids=df_es.doc_id.values.tolist())

-- -- Encoding corpus...
-- -- Corpus encoded in 25.68424938519796 minutes


In [49]:
query = "¿Cómo puedo ayudar a mi bebé a dormir?"#"How can I help my baby to sleep?"
results = index_all_es.retrieve(query, topk=10)

for result in results:
    print(f"Document ID: {result['document_id']}")
    print(f"Sentence: {result['sentence']}")
    print(f"Score: {result['score']}")
    print(f"Original Document: {result['original_document']}")
    print("\n")

Document ID: ES_923254_121494-4
Sentence: Esto puede despertar a un bebé dormido
Score: 0.8027505278587341
Original Document: Si la frecuencia cardíaca no sube por sí sola, le pueden solicitar que frote la mano sobre su vientre. Esto puede despertar a un bebé dormido. También se puede usar un dispositivo para enviar un ruido hacia el vientre. Esto no causará ningún dolor.


Document ID: ES_790477_103134-18
Sentence: ¿Cómo puede ayudar al bebé a quedarse dormido: No todos los bebés saben cómo dormirse solos
Score: 0.8019784092903137
Original Document: ¿Cómo puede ayudar al bebé a quedarse dormido: No todos los bebés saben cómo dormirse solos. Cuando es la hora de dormir, muchos padres mecen al bebé. Los recién nacidos y bebés pequeños se duermen mientras están amamantando. Tener una rutina para la hora de dormir es una buena idea. Pero si un bebé más grande se duerme mientras come o en brazos, puede convertirse en un patrón. Puede que el bebé empiece a esperar estar en brazos para dormi

In [52]:
df_es[df_es.doc_id == "ES_765361_101395-7"]

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang,thetas,top_k,main_topic,counts
1005581,1005581,precauciones_arropar bebé arropar significar envolver nacido firmeza_manta evitar mover_brazo pierna bebé sentar seguro ayudar él dormir comprar manta especial arropar_diseñado fácil envolver bebé,precaution_tuck baby wrapping mean wrap newborn blanket prevent move arm_leg information technology baby feel safe help sleep buy special blanket_wrap design information technology easy wrap baby,ES_765361_101395-7,Precauciones para arropar al bebé: Arropar significa envolver a los recién nacidos con firmeza en una manta para evitar que muevan los brazos y piernas. Puede hacer que un bebé se sienta seguro y ayudarlo a dormir. Usted puede comprar una manta especial para arropar diseñada especialmente para que sea más fácil envolver al bebé.,Precautions for tucking the baby: Wrapping means firmly wrapping the newborns in a blanket to prevent them from moving their arms and legs. It can make a baby feel safe and help him sleep. You can buy a special blanket to wrap specially designed to make it easier to wrap the baby.,ES,"[0.0, 0.0, 0.5, 0.2777777910232544, 0.0, 0.0555555559694767, 0.0, 0.0, 0.0, 0.0555555559694767, 0.0555555559694767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0555555559694767, 0.0]","[(2, 0.5), (3, 0.2777777910232544)]",2,20
